In [1]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     |████████████████████████████████| 616 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 37.9 MB/s 
     |████████████████████████████████| 237 kB 43.5 MB/s 
     |████████████████████████████████| 1.1 MB 34.4 MB/s 
     |████████████████████████████████| 840 kB 40.5 MB/s 
     |████████████████████████████████| 1.2 MB 28.1 MB/s 
     |████████████████████████████████| 6.4 MB 22.0 MB/s 
     |████████████████████████████████| 120 kB 48.8 MB/s 
     |████████████████████████████████| 3.4 MB 38.3 MB/s 
     |████████████████████████████████| 77 kB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 36.9 MB/s 
     |████████████████████████████████| 87 kB 6.2 MB/s 
     |████████████████████████████████| 25.3 MB 1.7 MB/s 
     |████████████████████████████████| 47.8 MB 1.8 MB/s 
     |████████████████████████████████| 352 kB 43.5 MB/s 
     |████████████████████████████████| 99 kB 8.9 MB/s 
     |████████████████████████████████| 462 kB 35.0 MB/s 
     |███████████████

In [2]:
import numpy as np
import os

In [3]:
from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [4]:
!wget https://github.com/anwaypimpalkar/picoboo-thermal-imaging-surveillance-system/raw/main/data/human-thermal-dataset.zip
!unzip -q human-thermal-dataset.zip

--2022-04-13 07:32:23--  https://github.com/anwaypimpalkar/picoboo-thermal-imaging-surveillance-system/raw/main/data/human-thermal-dataset.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/anwaypimpalkar/picoboo-thermal-imaging-surveillance-system/main/data/human-thermal-dataset.zip [following]
--2022-04-13 07:32:23--  https://raw.githubusercontent.com/anwaypimpalkar/picoboo-thermal-imaging-surveillance-system/main/data/human-thermal-dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40972922 (39M) [application/zip]
Saving to: ‘human-thermal-dataset.zip’

human-thermal-datas 100%[====

In [5]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'human-thermal-dataset/train',
    'human-thermal-dataset/train',
    ['human']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'human-thermal-dataset/validate',
    'human-thermal-dataset/validate',
    ['human']
)

In [6]:
train_data.size

388

In [7]:
val_data.size

18

In [8]:
spec = model_spec.get('efficientdet_lite2')

In [9]:
model = object_detector.create(train_data, model_spec=spec, batch_size=16, train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/20
24/24 [==============================] - 110s 2s/step - det_loss: 1.4404 - cls_loss: 0.9343 - box_loss: 0.0101 - reg_l2_loss: 0.0759 - loss: 1.5163 - learning_rate: 0.0140 - gradient_norm: 2.2853 - val_det_loss: 1.1238 - val_cls_loss: 0.6617 - val_box_loss: 0.0092 - val_reg_l2_loss: 0.0759 - val_loss: 1.1997
Epoch 2/20
24/24 [==============================] - 43s 2s/step - det_loss: 0.6731 - cls_loss: 0.3953 - box_loss: 0.0056 - reg_l2_loss: 0.0760 - loss: 0.7490 - learning_rate: 0.0197 - gradient_norm: 3.8406 - val_det_loss: 0.8188 - val_cls_loss: 0.4842 - val_box_loss: 0.0067 - val_reg_l2_loss: 0.0761 - val_loss: 0.8949
Epoch 3/20
24/24 [==============================] - 42s 2s/step - det_loss: 0.5192 - cls_loss: 0.3010 - box_loss: 0.0044 - reg_l2_loss: 0.0762 - loss: 0.5954 - learning_rate: 0.0191 - gradient_norm: 3.7748 - val_det_loss: 0.6721 - val_cls_loss: 0.4151 - val_box_loss: 0.0051 - val_reg_l2_loss: 0.0763 - val_loss: 0.7484
Epoch 4/20
24/24 [=====================

In [10]:
model.evaluate(val_data)

1/1 [==============================] - 8s 8s/step



{'AP': 0.75306594,
 'AP50': 0.9970297,
 'AP75': 0.91438556,
 'AP_/human': 0.75306594,
 'APl': 0.7883744,
 'APm': 0.2025,
 'APs': -1.0,
 'ARl': 0.84444445,
 'ARm': 0.3,
 'ARmax1': 0.5736842,
 'ARmax10': 0.81578946,
 'ARmax100': 0.81578946,
 'ARs': -1.0}

In [18]:
model.export(export_dir='.', tflite_filename='human_thermal_efflite2.tflite')

In [19]:
model.evaluate_tflite('human_thermal_efflite2.tflite', val_data)

18/18 [==============================] - 165s 9s/step



{'AP': 0.7267763,
 'AP50': 0.9970297,
 'AP75': 0.93584985,
 'AP_/human': 0.7267763,
 'APl': 0.7535018,
 'APm': 0.22777778,
 'APs': -1.0,
 'ARl': 0.7888889,
 'ARm': 0.7,
 'ARmax1': 0.5473684,
 'ARmax10': 0.7842105,
 'ARmax100': 0.7842105,
 'ARs': -1.0}

In [20]:
# Download the TFLite model to your local computer.
from google.colab import files
files.download('human_thermal_efflite2.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>